In [ ]:
import gradio as gr
import os
import tempfile
import numpy as np
import soundfile as sf
from dotenv import load_dotenv
from openai import OpenAI
import random

load_dotenv()
client = OpenAI()

languages = [
    'Afrikaans', 'Albanian', 'Amharic', 'Arabic', 'Armenian', 'Azerbaijani', 'Basque', 'Belarusian', 
    'Bengali', 'Bosnian', 'Bulgarian', 'Catalan', 'Cebuano', 'Chichewa', 'Chinese', 'Corsican', 
    'Croatian', 'Czech', 'Danish', 'Dutch', 'English', 'Esperanto', 'Estonian', 'Filipino', 'Finnish', 
    'French', 'Galician', 'Georgian', 'German', 'Greek', 'Gujarati', 'Haitian Creole', 'Hausa', 
    'Hawaiian', 'Hebrew', 'Hindi', 'Hmong', 'Hungarian', 'Icelandic', 'Igbo', 'Indonesian', 
    'Irish', 'Italian', 'Japanese', 'Javanese', 'Kannada', 'Kazakh', 'Khmer', 'Kinyarwanda', 
    'Korean', 'Kurdish', 'Kyrgyz', 'Lao', 'Latin', 'Latvian', 'Lithuanian', 'Luxembourgish', 
    'Macedonian', 'Malagasy', 'Malay', 'Malayalam', 'Maltese', 'Maori', 'Marathi', 'Mongolian', 
    'Myanmar (Burmese)', 'Nepali', 'Norwegian', 'Odia', 'Pashto', 'Persian', 'Polish', 'Portuguese', 
    'Punjabi', 'Romanian', 'Russian', 'Samoan', 'Scots Gaelic', 'Serbian', 'Sesotho', 'Shona', 
    'Sindhi', 'Sinhala', 'Slovak', 'Slovenian', 'Somali', 'Spanish', 'Sundanese', 'Swahili', 
    'Swedish', 'Tajik', 'Tamil', 'Tatar', 'Telugu', 'Thai', 'Turkish', 'Turkmen', 'Ukrainian', 
    'Urdu', 'Uyghur', 'Uzbek', 'Vietnamese', 'Welsh', 'Xhosa', 'Yiddish', 'Yoruba', 'Zulu'
]

languages = ['Danish']

def translate_word(word, source_lang="da", target_lang="en"):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": f"""You are a translator from {source_lang} to {target_lang}. 
             Your output should be similar to a dictionary definition.
             Include plural cases, verb forms, genders, cases and other relevant information."""},
            {"role": "user", "content": f"Translate the word: {word}"}
        ]
    )
    return response.choices[0].message.content.strip()

def generate_sentence(word, language="en"):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": f"You are a sentence generator in {language}."},
            {"role": "user", "content": f"Generate a simple sentence using the word: {word}"}
        ]
    )
    return response.choices[0].message.content.strip()

def add_silence(input_file, output_file, silence_duration_sec=0.5):
    data, samplerate = sf.read(input_file)
    silence_samples = int(silence_duration_sec * samplerate)
    silence = np.zeros((silence_samples, data.shape[1])) if data.ndim > 1 else np.zeros(silence_samples)
    new_data = np.concatenate((silence, data), axis=0)
    sf.write(output_file, new_data, samplerate)

def get_audio(text, lang="da"):
    voices = ["alloy", "echo", "fable", "onyx", "nova", "shimmer"]
    selected_voice = random.choice(voices)
    prompt = f"Følgende tekst er dansk: '{text}'"

    with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3") as temp_audio_file:
        temp_audio_path = temp_audio_file.name
        with client.audio.speech.with_streaming_response.create(
            model="tts-1",
            voice=selected_voice,
            input=prompt,
        ) as response:
            response.stream_to_file(temp_audio_path)

    output_file_path = f"audio_{text}.mp3"
    add_silence(temp_audio_path, output_file_path, silence_duration_sec=1)
    os.remove(temp_audio_path)
    return output_file_path

def save_into_database(word, translation, sentence, audio_path):
    # Implement your database saving logic here
    print(f"Saving to database: {word}, {translation}, {sentence}, {audio_path}")
    return "Data saved successfully"

def process_word(danish_word, custom_sentence="", language=""):
    translation = translate_word(danish_word, language)
    sentence = custom_sentence if custom_sentence else generate_sentence(translation, language)
    audio_path = get_audio(danish_word, language)
    
    return translation, sentence, audio_path

def regenerate_sentence(word, language):
    return generate_sentence(word, language)

def save_to_database(danish_word, translation, sentence, audio_path):
    return save_into_database(danish_word, translation, sentence, audio_path)

# Gradio interface
with gr.Blocks() as iface:
    gr.Markdown("# Danish-English Language Learning App")
    gr.Markdown("Enter a Danish word to get its English translation, a sample sentence, and pronunciation.")
    
    with gr.Row():
        danish_word = gr.Textbox(label="Danish Word")
        language = gr.Dropdown(choices=languages, value="English", label="Target Language")
    
    process_btn = gr.Button("Process Word")
    
    with gr.Column():
        translation = gr.Textbox(label="Translation")
        sentence = gr.Textbox(label="Generated Sentence", interactive=True)
        audio = gr.Audio(label="Pronunciation")
    
    regenerate_btn = gr.Button("Regenerate Sentence")
    save_btn = gr.Button("Save to Database")
    save_result = gr.Textbox(label="Database Save Result")
    
    def process_and_display(word, lang):
        trans, sent, audio_path = process_word(word, language=lang)
        return trans, sent, audio_path
    
    process_btn.click(
        process_and_display,
        inputs=[danish_word, language],
        outputs=[translation, sentence, audio]
    )
    
    regenerate_btn.click(
        regenerate_sentence,
        inputs=[translation, language],
        outputs=sentence
    )
    
    save_btn.click(
        save_to_database,
        inputs=[danish_word, translation, sentence, audio],
        outputs=save_result
    )

if __name__ == "__main__":
    iface.launch()

In [ ]:
import gradio as gr
import io
import tempfile
import os

def generate_binary_file():
    # This is an example function that creates binary data
    # Replace this with your actual binary file generation logic
    return b"Hello, this is sample binary content!"

def download_file():
    # Get the binary data
    binary_data = generate_binary_file()
    
    # Create a temporary file to store the binary data
    temp_dir = tempfile.mkdtemp()
    temp_path = os.path.join(temp_dir, "download.bin")
    
    # Write the binary data to the temporary file
    with open(temp_path, "wb") as f:
        f.write(binary_data)
    
    return temp_path

# Create the Gradio interface
with gr.Blocks() as demo:
    gr.Markdown("## Binary File Download Demo")
    download_button = gr.Button("Download File")
    file_output = gr.File(label="Downloaded File")
    
    download_button.click(
        fn=download_file,
        outputs=file_output,
        api_name="download"
    )

if __name__ == "__main__":
    demo.launch()

Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


/mnt/c/Users/ambru/OneDrive/Projects/anki-words-builder/.venv/lib/python3.12/site-packages/gradio/analytics.py:106: UserWarning: IMPORTANT: You are using gradio version 4.42.0, however version 4.44.1 is available, please upgrade. 
--------
  warnings.warn(
